In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import re
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
기사18 = pd.read_csv("카카오기사.csv", encoding="cp949")
주식18 = pd.read_csv("카카오증권.csv")
카카오검증기사 = pd.read_csv("카카오기사_검증.csv", encoding="cp949")

In [4]:
기사18 = 기사18.drop(["뉴스 식별자"], axis =1)

In [5]:
기사18

,일자,키워드
0,20181231,"본격화,보급,헬스케어,성장,클라우드,엔터,자율,바이오,유망,도전,주식,시장,성장세,..."
1,20181231,"음식점,업주,배달,매출,수익,개선,긍정적,소상공인연합회,음식점,업주,대상,설문,배달..."
2,20181231,"김현미,새해,안전,편안,일상,성장,노동자,안전사각지대,출범,광역,교통위,상반기,광역..."
3,20181231,"경제,정책,튜닝,정부,수혜주,내수,초점,새해,경제,정책,변동성,수출주,정책,수혜,가..."
4,20181231,"음원,사용료,인상,결제,멜론,지니뮤직,고객,인상,새해,저작권료,음원,가격,인상,음원..."
...,...,...
8759,20180102,"카카오,3.65%,지속,체결,강도,강세,219%,카카오,2일,3.65%,142,00..."
8760,20180102,"삼익가구,삼익동계올림픽,올림픽,개최,기념,삼익,동계,올림픽,가구,세일,이벤트,진행,..."
8761,20180102,"투자,유망,종목,삼성전자,CJ,E&M,업종,바이오,삼성전자,CJ,E&M,코스피,코스..."
8762,20180102,"자녀,주식,연속,삼성전자,IL01,삼성전자,자녀,연속,주식,1위,이름,제약,바이오,..."


In [21]:
기사18.일자 = pd.to_datetime(기사18.일자, format ="%Y%m%d")
카카오검증기사.일자 = pd.to_datetime(카카오검증기사.일자, format ="%Y%m%d")

In [6]:
주식18

,Date,Open,Close,주가변동
0,2018. 1. 3 오후 3:30:00,30100,29800,-300
1,2018. 1. 4 오후 3:30:00,30500,31200,700
2,2018. 1. 5 오후 3:30:00,31400,31200,-200
3,2018. 1. 8 오후 3:30:00,32100,31900,-200
4,2018. 1. 9 오후 3:30:00,31600,31400,-200
...,...,...,...,...
238,2018. 12. 21 오후 3:30:00,20500,20900,400
239,2018. 12. 24 오후 3:30:00,20900,20200,-700
240,2018. 12. 26 오후 3:30:00,19800,19700,-100
241,2018. 12. 27 오후 3:30:00,20100,20300,200


In [7]:
주식18.Date = 주식18.Date.apply(lambda x: x.split("오후")[0])

In [8]:
for i in range(0,243):
    주식18.Date[i] = 주식18.Date[i].replace(". ","").replace(" ","")
주식18.Date = pd.to_datetime(주식18.Date, format ="%Y%m%d")

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
주식18.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    243 non-null    datetime64[ns]
 1   Open    243 non-null    int64         
 2   Close   243 non-null    int64         
 3   주가변동    243 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 7.7 KB


In [118]:
카카오검증기사.sort_values("일자")

,일자,키워드
12232,2020-01-02,"택시,면허,확보,벤티,마카롱,이동수단,경험,타다,카카오모빌리티,벤티,카니발,스타렉스..."
12215,2020-01-02,"YG,라스,조서윤,무한도전,제영재,PD,티캐스트로,YG엔터테인먼트,이적,MBC,출신..."
12216,2020-01-02,"콘텐츠,시대,네이버,네이버,오디오,클립,팟빵,오디오,콘텐츠,플랫폼,급성장,방송,인터..."
12217,2020-01-02,"저칼로리,프리미엄,아이스크림,인기,겨울,제맛,아이스크림,일반적,겨울,빙과,시장,대표..."
12218,2020-01-02,"끝장,오프라인,생존경쟁,vs,온라,새판,유통,업계,생존,예상,경기,침체,소비,심리,..."
...,...,...
23,2020-12-31,"며느라기,박하선X권율,달달,신혼,데이트,에피소드,예고,카카오,TV,공개,며느라기,7..."
22,2020-12-31,"디데이,어도비,플래시,지원,종료,즉시,삭제,권장,종료,1월,콘텐츠,플래시,차단,설치..."
21,2020-12-31,"페북,성범죄물,유튜브,불법,촬영,방통위,피해신고,디지털,성범죄,피해,신고,안내,삭제..."
28,2020-12-31,"감성,보컬,나훈아,반전,라이브,싱어게,전체공개,허스키,음색,감성,주목,싱어게인,감성..."


In [22]:
기사18 = 기사18.sort_values("일자")
카카오검증기사 = 카카오검증기사.sort_values("일자")

In [29]:
기사18_1 = 기사18.groupby("일자").키워드.apply(lambda x : ",".join(x)).reset_index()
카카오검증기사1 = 카카오검증기사.groupby("일자").키워드.apply(lambda x : ",".join(x)).reset_index()

In [12]:
for i,string in enumerate(카카오검증기사1['키워드']):
    카카오검증기사1['키워드'][i] =  re.sub(r'[0-9]+', '', string)

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
for i,string in enumerate(기사18_1['키워드']):
    기사18_1['키워드'][i] =  re.sub(r'[0-9]+', '', string)

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


종가와 전일종가의 차이가 0, 양수면 수치컬럼을 1로 지정, 음수면 -1로 지정.

In [16]:
주식18["수치"]= 주식18.주가변동.apply(lambda x : 1 if x >= 0 else -1)
주식18 = 주식18.rename(columns ={"Date":"일자"})

In [31]:
통합 = pd.merge(주식18,기사18_1)

In [32]:
통합

,일자,Open,Close,주가변동,수치,키워드
0,2018-01-03,30100,29800,-300,-1,"알짜,게임사,속도,카카오게임즈,상장,창립,기업,가치,성장,연내,코스닥,입성,추진,어..."
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카..."
2,2018-01-05,31400,31200,-200,-1,"유튜브,네이버,위,세금,만,순이용자수,구글,유튜브,동영,플랫폼,월평균,이용,시간,카..."
3,2018-01-08,32100,31900,-200,-1,"카카오,암호화,화폐,두나무,가치,반영,유진,유진투자증권,카카오,암호화,화폐,회사,나..."
4,2018-01-09,31600,31400,-200,-1,"셀트리온,상장,수혜주,셀트리온,코스닥,유가증권시장,코스피,상장,코스피,편입,시점,월..."
...,...,...,...,...,...,...
238,2018-12-21,20500,20900,400,1,"택시,생존,서비스경쟁,모임,연말,귀가시간,대화,택시,경험,기억,만취상태,바가지,승차..."
239,2018-12-24,20900,20200,-700,-1,"바이오,게임,PBR,저평가,우울,종목들,NHN엔터테인먼트,정체,네오위즈,실적,개선,..."
240,2018-12-26,19800,19700,-100,-1,"전일,순매도상위,거래소,기관,순매도,상위,종목,화학,업종,기관,투자자,거래소,현대건..."
241,2018-12-27,20100,20300,200,1,"크리에이터,꿀팁,유지어터,완벽,레시피,일,방송,다이어트,사람들,시선,특별,크리에이터..."


단어 극성구하기


 ![title](1.png)

식 (1)과 (2)에서 TermScore(i)는 어휘 i의 감성 점수이며, 0에서 1의 값을 가진다. TotalNum(i)는 뉴스 전체에서 나온 i의 출현 빈도이며, Num(i)∈PosDocs는 긍정적 영향을 
갖는 뉴스에서 발생한 i의 출현 빈도이다. 여기서 긍정적 영향을 갖는 뉴스는 각 종목의 주가가 상승한 날 발생한 뉴스를 의미한다. 즉, 주가가 상승한 날의 뉴스는 긍정적 의미가 있는 것으로 간주한다. 반대로 Num(i)∈NegDocs는 부정적 영향을 갖는 뉴스에서 발생한 i의 출현 빈도이다.

 ![title](2.png)

이러한 과정에서 주가가 상승한 날의 뉴스와 하락한 날의 뉴스에 동시에 출현하는 중복 어휘가 발생한다. 식 (3)과 같이 해당 어휘의 긍정점수와 부정점수를 합산하여 최종 감성 점수를 부여한다. 최종적으로 Termscore(i) > 0일 경우 긍정 속성단어로 분류하고, Termscore(i) <0일 경우에
는 부정 속성단어로 분류할 예정이다. 

단어의 빈도 생성

In [76]:
news = ','.join(통합.키워드.fillna('').values).lower().split(',')
news1 = [n for n in news if ((len(n) > 0) and (len(n) <15))]
통합수치 = Counter(news1)
len(통합수치)

63189

In [35]:
분할통합 = 통합.groupby("수치").키워드.apply(lambda x :",".join(x)).reset_index()
분할통합

,수치,키워드
0,-1,"알짜,게임사,속도,카카오게임즈,상장,창립,기업,가치,성장,연내,코스닥,입성,추진,어..."
1,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카..."


In [150]:
nouns = 분할통합[분할통합.수치 == -1].키워드.values[0].lower().split(',')
nouns = [n for n in nouns if ((len(n) > 0) and (len(n) <15))]
감소수치 = Counter(nouns)
nouns1 = 분할통합[분할통합.수치 == 1].키워드.values[0].lower().split(',')
nouns1 = [n for n in nouns1 if ((len(n) > 0) and (len(n) <15))]
증가수치 = Counter(nouns1)

In [151]:
len(증가수치)
len(감소수치)
증가수치.subtract(감소수치)
len(증가수치)

41853

45193

63189

In [152]:
증가수치.most_common(15)

[('.%', 1284),
 ('대비', 633),
 ('음성', 483),
 ('기록', 448),
 ('전년', 441),
 ('서비스', 404),
 ('매출', 396),
 ('증가', 394),
 ('분기', 345),
 ('적용', 322),
 ('스피커', 320),
 ('동기', 308),
 ('기아차', 307),
 ('블록체', 298),
 ('출시', 284)]

단어의 감성점수 생성 함수

In [119]:
def NounsFigure(Nouns):
    nouns = Nouns.lower().split(',')
    nouns = [n for n in nouns if ((len(n) > 0) and (len(n) <15))]
    SumFig =0
    for j in nouns:
        if 증가수치.get(j) is not None:
            SumFig = SumFig + (증가수치.get(j)/통합수치.get(j))
    return SumFig

In [43]:
통합.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      243 non-null    datetime64[ns]
 1   Open    243 non-null    int64         
 2   Close   243 non-null    int64         
 3   주가변동    243 non-null    int64         
 4   수치      243 non-null    int64         
 5   키워드     243 non-null    object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 13.3+ KB


In [44]:
len(통합[통합.수치 == 1])
len(통합[통합.수치 == -1])

120

123

주가의 증가횟수랑 감소횟수가 비슷한 것을 알 수 있다.

In [47]:
통합["지수"] = 0
for i,string in enumerate(통합['키워드']):
    통합['지수'][i] =  NounsFigure(string)

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


일별 기사에 등장하는 감성점수의 총합을 '지수'컬럼에 저장하여  지수가 0보다 크면 기사의 긍정적인 영향을 받아 주가도 상승할 것이라 추정,
반대로 지수가 0보다 작으면 주가가 하락할것이라 추정.

In [49]:
통합[통합.수치 ==1]
통합[((통합.지수 >= 0) &(통합.수치 == 1))]

,일자,Open,Close,주가변동,수치,키워드,지수
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카...",442
7,2018-11-01,18400,18620,220,1,"제,연쇄,창업,성공,신화,소프트뱅크벤처스,사단,블록체,혁신,심장,벤처캐피탈,출발,대...",-61
9,2018-11-02,19040,19640,600,1,"유비케어,억,영업,이익,전년,.%,유비케어,헬스,케어,전문,기업,영업이익,연결,영업...",-105
10,2018-11-05,28300,29000,700,1,"SPC,그룹,인천공항공사,장애인,일자리,협약,제결,CJ,푸드빌,빕스,G마켓,빅스마일...",-159
13,2018-11-06,19140,19320,180,1,"카카오,공유,창작자,경험,크리에이터스데이,연다,브런치,다음웹툰,카카오,이모티콘,카카...",-139
...,...,...,...,...,...,...,...
234,2018-12-17,22000,22100,100,1,"KBS미디어,블록체,콘텐츠서비스,활용,센티넬,프로토콜,사이버,보안,솔루션,블록체,플...",492
235,2018-12-18,21700,21700,0,1,"곳,상장,폐지,기업,전년,%,감소,국내증시,상장,폐지,기업,동기,%가량,한국,거래소...",440
238,2018-12-21,20500,20900,400,1,"택시,생존,서비스경쟁,모임,연말,귀가시간,대화,택시,경험,기억,만취상태,바가지,승차...",280
241,2018-12-27,20100,20300,200,1,"크리에이터,꿀팁,유지어터,완벽,레시피,일,방송,다이어트,사람들,시선,특별,크리에이터...",437


,일자,Open,Close,주가변동,수치,키워드,지수
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카...",442
16,2018-11-08,26900,27100,200,1,"유한킴벌리,딸들,생리대,기부,중저가,제품,출시,취약계층,황사,마스크,전달,유한킴벌리...",1639
17,2018-11-08,19580,20000,420,1,"유한킴벌리,딸들,생리대,기부,중저가,제품,출시,취약계층,황사,마스크,전달,유한킴벌리...",1639
18,2018-11-09,27200,27600,400,1,"제네시스,EQ,G,재탄생,명칭,레터링,엠블럼,통일,풀체,부분변경,내비,자동,무선,업...",473
19,2018-11-09,20000,20100,100,1,"제네시스,EQ,G,재탄생,명칭,레터링,엠블럼,통일,풀체,부분변경,내비,자동,무선,업...",473
...,...,...,...,...,...,...,...
234,2018-12-17,22000,22100,100,1,"KBS미디어,블록체,콘텐츠서비스,활용,센티넬,프로토콜,사이버,보안,솔루션,블록체,플...",492
235,2018-12-18,21700,21700,0,1,"곳,상장,폐지,기업,전년,%,감소,국내증시,상장,폐지,기업,동기,%가량,한국,거래소...",440
238,2018-12-21,20500,20900,400,1,"택시,생존,서비스경쟁,모임,연말,귀가시간,대화,택시,경험,기억,만취상태,바가지,승차...",280
241,2018-12-27,20100,20300,200,1,"크리에이터,꿀팁,유지어터,완벽,레시피,일,방송,다이어트,사람들,시선,특별,크리에이터...",437


18년도 기사의 단어로 감성사전을 만들고 18년도의 지수를 계산한 결과 120개중 115개을 맞추는 정확도를 보였다. 

In [50]:
카카오검증주식 = pd.read_csv("카카오증권검증.csv", encoding="cp949")
카카오검증주식

,Date,Open,Close,주가변동
0,2020. 1. 3 오후 3:30:00,30700,30500,-200
1,2020. 1. 6 오후 3:30:00,30100,30900,800
2,2020. 1. 7 오후 3:30:00,31000,32000,1000
3,2020. 1. 8 오후 3:30:00,31700,31900,200
4,2020. 1. 9 오후 3:30:00,32700,32800,100
...,...,...,...,...
242,2020. 12. 23 오후 3:30:00,76200,75500,-700
243,2020. 12. 24 오후 3:30:00,75700,74800,-900
244,2020. 12. 28 오후 3:30:00,74900,74600,-300
245,2020. 12. 29 오후 3:30:00,76200,76900,0


In [51]:
카카오검증주식.Date = 카카오검증주식.Date.apply(lambda x: x.split("오후")[0])
for i in range(0,247):
    카카오검증주식.Date[i] = 카카오검증주식.Date[i].replace(". ","").replace(" ","")
카카오검증주식.Date = pd.to_datetime(카카오검증주식.Date, format ="%Y%m%d")

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [52]:
카카오검증주식["수치"]= 카카오검증주식.주가변동.apply(lambda x : 1 if x >= 0 else -1)
카카오검증주식 = 카카오검증주식.rename(columns ={"Date":"일자"})

In [53]:
카카오검증통합 = pd.merge(카카오검증주식,카카오검증기사1)

In [54]:
카카오검증통합["지수"] = 0
for i,string in enumerate(카카오검증통합['키워드']):
    카카오검증통합['지수'][i] =  NounsFigure(string)

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
카카오검증통합

,일자,Open,Close,주가변동,수치,키워드,지수
0,2020-01-03,30700,30500,-200,-1,"미디어SR,대한민국,공익,법인,백서,발간,활동,사회,책임,조명,미디어SR,인터넷,경...",-170
1,2020-01-06,30100,30900,800,1,"5G,선도,CT,기업,종주국,위상,7일,현지시간,나흘간,미국,라스베이거스,컨벤션,센...",-159
2,2020-01-07,31000,32000,1000,1,"CES,금융,경쟁사,구글,회장,윤종규,KB,금융,라스베이거스,신한,CES,디지털,전...",-215
3,2020-01-08,31700,31900,200,1,"이동수단,음식배달,금융,모빌리티,일상,서울,강남,부산,거래처,출장,수서역,이동,차량...",-37
4,2020-01-09,32700,32800,100,1,"제주,청정생물자원,천연,면역,항암치료,물질,로컬,포커스,강소,기업,CEO,대표,제주...",-135
...,...,...,...,...,...,...,...
242,2020-12-23,76200,75500,-700,-1,"코스피,회복,달러,환율,상승,코스피지수,하루,상승,장중,회복,외국인,기관,물량,개인...",-516
243,2020-12-24,75700,74800,-900,-1,"최악,자영업,배달,시작,배달,대한민국,연말연시,전국,사실,셧다운,배달,비상,사회,거...",-489
244,2020-12-28,74900,74600,-300,-1,"연애혁명,박지훈,종영,소감,연애,혁명,순수,오랜만,고등학생,일상,경험,연애혁명,배우...",-495
245,2020-12-29,76200,76900,0,1,"솔루션,보안,언택트,수혜,포춘클럽,라운지,에스원,무인매장,재택근무,관리,솔루션,포스...",-227


In [56]:
카카오검증통합[카카오검증통합.수치 == 1]
카카오검증통합[(카카오검증통합.수치 == 1) & (카카오검증통합.지수 >= 0)]

,일자,Open,Close,주가변동,수치,키워드,지수
1,2020-01-06,30100,30900,800,1,"5G,선도,CT,기업,종주국,위상,7일,현지시간,나흘간,미국,라스베이거스,컨벤션,센...",-159
2,2020-01-07,31000,32000,1000,1,"CES,금융,경쟁사,구글,회장,윤종규,KB,금융,라스베이거스,신한,CES,디지털,전...",-215
3,2020-01-08,31700,31900,200,1,"이동수단,음식배달,금융,모빌리티,일상,서울,강남,부산,거래처,출장,수서역,이동,차량...",-37
4,2020-01-09,32700,32800,100,1,"제주,청정생물자원,천연,면역,항암치료,물질,로컬,포커스,강소,기업,CEO,대표,제주...",-135
5,2020-01-10,33000,33000,0,1,"금융,의료,통신,개인맞춤,개막,서비스,시대,개인정보,경제,사용,허용,데이터,입법,금...",-96
...,...,...,...,...,...,...,...
231,2020-12-04,74800,77900,3100,1,"LGU,수험생,BTS,시청,쏜다,LG,수고,수험,대상,프로모션,시행,LG유플러스,학...",-582
234,2020-12-11,74800,74900,100,1,"코스피,최고,장중,역대,경신,외국인,팔자,상승세,코스피,개장,2770선,개장,돌파,...",-411
240,2020-12-21,73400,75900,2500,1,"카카오뱅크,미니,고객,이용,청소년,기부,캠페인,카카오뱅크,이달,27일,고객,기부,응...",-436
245,2020-12-29,76200,76900,0,1,"솔루션,보안,언택트,수혜,포춘클럽,라운지,에스원,무인매장,재택근무,관리,솔루션,포스...",-227


,일자,Open,Close,주가변동,수치,키워드,지수
10,2020-11-05,33400,34000,600,1,"카카오,1조,분기,매출,특수,실적,사상,최대,상보,매출,1조,영업익,1202억,전년...",50
11,2020-11-05,71800,72600,800,1,"카카오,1조,분기,매출,특수,실적,사상,최대,상보,매출,1조,영업익,1202억,전년...",50
92,2020-05-07,39900,41200,1300,1,"사회,복지,시설,종사자,65%,직장,경험,10명,사회,복지,시설,종사자,6명,1년,...",73
157,2020-08-06,71700,72800,1100,1,"진격,카카오,광고,쇼핑,페이,웹툰,카카오,코로나19,역대,최대,매출,영업,이익,기록...",238


그러나 18년도의 감성사전으로 20년도의 기사단어를 통해 지수를 계산한 결과 정확도는 131개중 4개만 맞추는 정확도를 보인다. 그리하여 감성사전에 19년도 단어를 추가하기로 하였다.

In [57]:
증권19 = pd.read_csv("카카오19주가.csv", encoding="cp949")
기사19 = pd.read_csv("카카오19기사.csv", encoding="cp949")

In [58]:
기사19.일자 = pd.to_datetime(기사19.일자, format ="%Y%m%d")

In [59]:
기사19 = 기사19.sort_values("일자")
기사19 = 기사19.groupby("일자").키워드.apply(lambda x : ",".join(x)).reset_index()

In [60]:
기사19

,일자,키워드
0,2019-01-02,"은행,디지털,단계,한국,금융,시장,혁신,금리인상,무역전쟁,은행권,리스크,관리,디지털..."
1,2019-01-03,"AI,새해,주식시장,연일,약세,코스피,달여,2000선,코스닥,부진,전문가들,하강,글..."
2,2019-01-04,"신한은행,서울시금고,업무,시작,단축,오픈,예정,일정,시간,주목,104년,서울시금고,..."
3,2019-01-05,"진짜,초콜릿,2월,3월,대목,준비,쇼콜라티에,수제,초콜릿,기술자,분주,밸런타인데이,..."
4,2019-01-06,"카카오,묘수,네이버,증권사,인수,추진,편집자,기사,17:14,프리미엄,컨버전스,미디..."
...,...,...
357,2019-12-27,"10대,아주,경제,소프트웨어,구글,페이스북,ICT,동맹,트렌드로,업계,관통,핵심,트..."
358,2019-12-28,"마일리지,할인,대세,신용,카드,머니가족,50대,나머니,가족,나머니,일상생활,좌충우돌..."
359,2019-12-29,"해외,SI,입찰,참여,저울,로젠택배,예고,인수전,흥행,카카오,SK에너지,관심,편집자..."
360,2019-12-30,"일상속,주방로봇,디지털,혁신,본격화,매경,CES,디지털,일상,변화,시작,연초,개최,..."


In [61]:
증권19

,Date,Open,Close,주가변동
0,2019. 1. 3 오후 3:30:00,20500,20500,0
1,2019. 1. 4 오후 3:30:00,20400,20700,300
2,2019. 1. 7 오후 3:30:00,21000,20600,-400
3,2019. 1. 8 오후 3:30:00,20500,19980,-520
4,2019. 1. 9 오후 3:30:00,20100,20400,300
...,...,...,...,...
240,2019. 12. 23 오후 3:30:00,30200,29700,-500
241,2019. 12. 24 오후 3:30:00,29600,29300,-300
242,2019. 12. 26 오후 3:30:00,29200,29600,400
243,2019. 12. 27 오후 3:30:00,29900,30700,800


In [62]:
증권19.Date = 증권19.Date.apply(lambda x: x.split("오후")[0])
for i in range(0,245):
    증권19.Date[i] = 증권19.Date[i].replace(". ","").replace(" ","")
증권19.Date = pd.to_datetime(증권19.Date, format ="%Y%m%d")

c:\users\eoghk\anaconda3\envs\section\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [63]:
증권19["수치"]= 증권19.주가변동.apply(lambda x : 1 if x >= 0 else -1)
증권19 = 증권19.rename(columns ={"Date":"일자"})

In [64]:
통합19 = pd.merge(증권19,기사19)

In [128]:
통합19

,일자,Open,Close,주가변동,수치,키워드
0,2019-01-03,20500,20500,0,1,"AI,새해,주식시장,연일,약세,코스피,달여,선,코스닥,부진,전문가들,하강,글로벌,경..."
1,2019-01-04,20400,20700,300,1,"신한은행,서울시금고,업무,시작,단축,오픈,예정,일정,시간,주목,년,서울시금고,은행,..."
2,2019-01-07,21000,20600,-400,-1,"사장,정일문,한국투자증권,영업익,년,순익,사장,정일문,한국투자증권,신임,서울,여의도..."
3,2019-01-08,20500,19980,-520,-1,"KISO,부동산,매물,검증법,특허,출원,허위매물,허위매물,수신,검증,정보,갱신,단계..."
4,2019-01-09,20100,20400,300,1,"패스트파이브,회원,달성,네이버,카카오,패스트파이브,회원,명,달성,IT기업,대표,기업..."
...,...,...,...,...,...,...
240,2019-12-23,30200,29700,-500,-1,"실검,악용,악플,차단,네이버,카카오,서비스,핵심,포털,네이버,카카오,포털,업체,실시..."
241,2019-12-24,29600,29300,-300,-1,"택시,강남,종로,연말,특수,모빌리티,경쟁,연말연시,택시,수요,모빌리티,업체,치열,서..."
242,2019-12-26,29200,29600,400,1,"마케팅대상,대한민국,마케팅,대상,모바일마케팅,대상,GS건설,자이,SNS채널,건설업,..."
243,2019-12-27,29900,30700,800,1,"대,아주,경제,소프트웨어,구글,페이스북,ICT,동맹,트렌드로,업계,관통,핵심,트렌드..."


In [129]:
for i,string in enumerate(통합19['키워드']):
    통합19['키워드'][i] =  re.sub(r'[0-9]+', '', string)

In [132]:
통합1819 = pd.concat([통합,통합19])

In [133]:
통합1819

,일자,Open,Close,주가변동,수치,키워드,지수
0,2018-01-03,30100,29800,-300,-1,"알짜,게임사,속도,카카오게임즈,상장,창립,기업,가치,성장,연내,코스닥,입성,추진,어...",-716.0
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카...",442.0
2,2018-01-05,31400,31200,-200,-1,"유튜브,네이버,위,세금,만,순이용자수,구글,유튜브,동영,플랫폼,월평균,이용,시간,카...",-455.0
3,2018-01-08,32100,31900,-200,-1,"카카오,암호화,화폐,두나무,가치,반영,유진,유진투자증권,카카오,암호화,화폐,회사,나...",-965.0
4,2018-01-09,31600,31400,-200,-1,"셀트리온,상장,수혜주,셀트리온,코스닥,유가증권시장,코스피,상장,코스피,편입,시점,월...",-398.0
...,...,...,...,...,...,...,...
240,2019-12-23,30200,29700,-500,-1,"실검,악용,악플,차단,네이버,카카오,서비스,핵심,포털,네이버,카카오,포털,업체,실시...",NaN
241,2019-12-24,29600,29300,-300,-1,"택시,강남,종로,연말,특수,모빌리티,경쟁,연말연시,택시,수요,모빌리티,업체,치열,서...",NaN
242,2019-12-26,29200,29600,400,1,"마케팅대상,대한민국,마케팅,대상,모바일마케팅,대상,GS건설,자이,SNS채널,건설업,...",NaN
243,2019-12-27,29900,30700,800,1,"대,아주,경제,소프트웨어,구글,페이스북,ICT,동맹,트렌드로,업계,관통,핵심,트렌드...",NaN


In [134]:
news1819 = ','.join(통합1819.키워드.fillna('').values).lower().split(',')
nouns1819 = [n for n in news1819 if ((len(n) > 0) and (len(n) <15))]
통합수치1819 = Counter(nouns1819)

In [135]:
len(통합수치1819)

98016

In [136]:
분할통합1819 = 통합1819.groupby("수치").키워드.apply(lambda x :",".join(x)).reset_index()
분할통합1819

,수치,키워드
0,-1,"알짜,게임사,속도,카카오게임즈,상장,창립,기업,가치,성장,연내,코스닥,입성,추진,어..."
1,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카..."


In [137]:
nouns = 분할통합1819[분할통합1819.수치 == -1].키워드.values[0].lower().split(',')
nouns = [n for n in nouns if ((len(n) > 0) and (len(n) <15))]
감소수치1819 = Counter(nouns)
nouns1 = 분할통합1819[분할통합1819.수치 == 1].키워드.values[0].lower().split(',')
nouns1 = [n for n in nouns1 if ((len(n) > 0) and (len(n) <15))]
증가수치1819 = Counter(nouns1)

In [138]:
len(증가수치1819)
len(감소수치1819)
증가수치1819.subtract(감소수치1819)
len(증가수치1819)

67233

68517

98016

In [153]:
증가수치1819.most_common(15)

[('.%', 1652),
 ('블록체', 907),
 ('클레이튼', 772),
 ('기록', 671),
 ('ai', 587),
 ('음성', 536),
 ('플랫폼', 527),
 ('종목', 523),
 ('출시', 507),
 ('모빌리티', 456),
 ('스피커', 437),
 ('만', 436),
 ('제공', 428),
 ('광고', 412),
 ('블록체인', 403)]

In [140]:
def NounsFigure1819(Nouns):
    nouns = Nouns.lower().split(',')
    nouns = [n for n in nouns if ((len(n) > 0) and (len(n) <15))]
    SumFig =0
    for j in nouns:
        if 증가수치1819.get(j) is not None:
            SumFig = SumFig + (증가수치1819.get(j)/통합수치1819.get(j))
    return SumFig

In [141]:
통합1819["지수"] = 0.0
len(통합1819[통합1819.수치 == 1])
len(통합1819[통합1819.수치 == -1])

254

234

In [142]:
for i,string in enumerate(통합1819['키워드']):
    통합1819['지수'][i] =  NounsFigure1819(string)

In [144]:
통합1819[통합1819.수치 ==1]
통합1819[(통합1819.수치 ==1) & (통합1819.지수 >= 0)]

,일자,Open,Close,주가변동,수치,키워드,지수
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카...",495.550786
7,2018-11-01,18400,18620,220,1,"제,연쇄,창업,성공,신화,소프트뱅크벤처스,사단,블록체,혁신,심장,벤처캐피탈,출발,대...",63.125577
9,2018-11-02,19040,19640,600,1,"유비케어,억,영업,이익,전년,.%,유비케어,헬스,케어,전문,기업,영업이익,연결,영업...",99.373768
10,2018-11-05,28300,29000,700,1,"SPC,그룹,인천공항공사,장애인,일자리,협약,제결,CJ,푸드빌,빕스,G마켓,빅스마일...",12.781183
13,2018-11-06,19140,19320,180,1,"카카오,공유,창작자,경험,크리에이터스데이,연다,브런치,다음웹툰,카카오,이모티콘,카카...",-21.489120
...,...,...,...,...,...,...,...
235,2019-12-16,30200,30300,100,1,"카뱅,독주,인터넷,은행,격변,예고,제인터넷,은행,토스,토스뱅크,주자,등장,인터넷,은...",498.885571
237,2019-12-18,30100,30200,100,1,"무협,한국,스웨덴,비즈니스,서밋,개최,김영주,회장,오른쪽,비즈니스스웨덴,일바,베리,...",-2627.953988
239,2019-12-20,30000,30000,0,1,"카페,동반성장,데이,파트너스,성료,글로벌,전자,상거래,플랫폼,업체,카페,전자,상거래...",-545.165743
242,2019-12-26,29200,29600,400,1,"마케팅대상,대한민국,마케팅,대상,모바일마케팅,대상,GS건설,자이,SNS채널,건설업,...",369.583003


,일자,Open,Close,주가변동,수치,키워드,지수
1,2018-01-04,30500,31200,700,1,"카카오톡,문자,메시지,확인,추가,일상,카카오톡,전략,카카오톡,만능,플랫폼,실행,카카...",495.550786
7,2018-11-01,18400,18620,220,1,"제,연쇄,창업,성공,신화,소프트뱅크벤처스,사단,블록체,혁신,심장,벤처캐피탈,출발,대...",63.125577
9,2018-11-02,19040,19640,600,1,"유비케어,억,영업,이익,전년,.%,유비케어,헬스,케어,전문,기업,영업이익,연결,영업...",99.373768
10,2018-11-05,28300,29000,700,1,"SPC,그룹,인천공항공사,장애인,일자리,협약,제결,CJ,푸드빌,빕스,G마켓,빅스마일...",12.781183
16,2018-11-08,26900,27100,200,1,"유한킴벌리,딸들,생리대,기부,중저가,제품,출시,취약계층,황사,마스크,전달,유한킴벌리...",1219.165623
...,...,...,...,...,...,...,...
225,2019-11-25,30700,31300,600,1,"카카오,라이언,런던,유럽,시장,진출,윈터,원더랜드,일,내년,운영,카카오,프렌즈,영국...",433.226221
234,2019-12-13,30100,30200,100,1,"대세,펭수,인기,관련주,EBS,펭수,펭귄,캐릭터,웹시트콤,진출,인기,관련주,주가,한...",522.183746
235,2019-12-16,30200,30300,100,1,"카뱅,독주,인터넷,은행,격변,예고,제인터넷,은행,토스,토스뱅크,주자,등장,인터넷,은...",498.885571
242,2019-12-26,29200,29600,400,1,"마케팅대상,대한민국,마케팅,대상,모바일마케팅,대상,GS건설,자이,SNS채널,건설업,...",369.583003


18,19년도 기사의 단어를 통해 감성사전을 만들고 그것을 통해 1819의 지수를 구해 보면 254개중 183개를 맞추는 정확도를 보인다.

In [145]:
카카오검증통합["지수"] = 0
for i,string in enumerate(카카오검증통합['키워드']):
    카카오검증통합['지수'][i] =  NounsFigure1819(string)

In [147]:
카카오검증통합[카카오검증통합.수치 == 1]
카카오검증통합[(카카오검증통합.수치 == 1) & (카카오검증통합.지수 >= 0)]

,일자,Open,Close,주가변동,수치,키워드,지수
1,2020-01-06,30100,30900,800,1,"5G,선도,CT,기업,종주국,위상,7일,현지시간,나흘간,미국,라스베이거스,컨벤션,센...",-44
2,2020-01-07,31000,32000,1000,1,"CES,금융,경쟁사,구글,회장,윤종규,KB,금융,라스베이거스,신한,CES,디지털,전...",-39
3,2020-01-08,31700,31900,200,1,"이동수단,음식배달,금융,모빌리티,일상,서울,강남,부산,거래처,출장,수서역,이동,차량...",-21
4,2020-01-09,32700,32800,100,1,"제주,청정생물자원,천연,면역,항암치료,물질,로컬,포커스,강소,기업,CEO,대표,제주...",29
5,2020-01-10,33000,33000,0,1,"금융,의료,통신,개인맞춤,개막,서비스,시대,개인정보,경제,사용,허용,데이터,입법,금...",76
...,...,...,...,...,...,...,...
231,2020-12-04,74800,77900,3100,1,"LGU,수험생,BTS,시청,쏜다,LG,수고,수험,대상,프로모션,시행,LG유플러스,학...",-211
234,2020-12-11,74800,74900,100,1,"코스피,최고,장중,역대,경신,외국인,팔자,상승세,코스피,개장,2770선,개장,돌파,...",-57
240,2020-12-21,73400,75900,2500,1,"카카오뱅크,미니,고객,이용,청소년,기부,캠페인,카카오뱅크,이달,27일,고객,기부,응...",69
245,2020-12-29,76200,76900,0,1,"솔루션,보안,언택트,수혜,포춘클럽,라운지,에스원,무인매장,재택근무,관리,솔루션,포스...",76


,일자,Open,Close,주가변동,수치,키워드,지수
4,2020-01-09,32700,32800,100,1,"제주,청정생물자원,천연,면역,항암치료,물질,로컬,포커스,강소,기업,CEO,대표,제주...",29
5,2020-01-10,33000,33000,0,1,"금융,의료,통신,개인맞춤,개막,서비스,시대,개인정보,경제,사용,허용,데이터,입법,금...",76
10,2020-11-05,33400,34000,600,1,"카카오,1조,분기,매출,특수,실적,사상,최대,상보,매출,1조,영업익,1202억,전년...",43
11,2020-11-05,71800,72600,800,1,"카카오,1조,분기,매출,특수,실적,사상,최대,상보,매출,1조,영업익,1202억,전년...",43
17,2020-12-01,73900,74900,1000,1,"카카오,구독서비스,성장사업,41만,유지,유안타증권,리포트,유안타증권,카카오,구독,서...",16
...,...,...,...,...,...,...,...
221,2020-11-17,71600,72400,800,1,"박하선,스마트폰,알고리즘,배우,박하선,요즘엄마,면모,17일,7시,공개,토크쇼,카카오...",121
222,2020-11-18,72000,73100,1100,1,"쇼핑,콘텐츠,잭팟,사업,네이버,카카오,최대,호황,국민,포털,메신저,기업,네이버,카카...",132
226,2020-11-24,73300,74600,1300,1,"세븐일레븐,천마표시멘트팝콘,이색,상품,출시,세븐일레븐,유동골뱅이맥주,시멘트,브랜드,...",128
240,2020-12-21,73400,75900,2500,1,"카카오뱅크,미니,고객,이용,청소년,기부,캠페인,카카오뱅크,이달,27일,고객,기부,응...",69


18,19년도 기사를 이용하여 20년도 지수를 구한 결과 131개 중 77개를 맞추는 정확도를 보인다.